### Readme:

This notebook documents what I learnt from https://course.spacy.io/en/

Special thanks to the content creators and the presenter Ines

If you want to learn more about spaCy, please visit https://spacy.io/ or https://course.spacy.io/en/

Thank you!

### Chapter 2: Large-scale data analysis with spaCy

Vocab : stores data shared across multiple documents

Strings are encoded to hash values

Strings are only stored once in the StringStore via
nlp.vocab.strings

Strings store : lookup table in both directions

staCy internally looks at hash id

Hashes can't be reversed

In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [7]:
coffee_hash = nlp.vocab.strings["coffee"]

In [8]:
coffee_hash # hash value

3197928453018144401

In [9]:
coffee_string = nlp.vocab.strings[coffee_hash]

In [10]:
coffee_string # string value

'coffee'

In [11]:
string = coffee_string = nlp.vocab.strings[319786] # errors if never see the strings before

KeyError: "[E018] Can't retrieve string for hash '319786'. This usually refers to an issue with the `Vocab` or `StringStore`."

In [12]:
# lexeme doesn't have context-depended pos, dependencies, or entity labels

In [13]:
from spacy.lang.en import English

nlp = English()
doc = nlp("I have a cat")

# Look up the hash for the word "cat"
cat_hash = nlp.vocab.strings["cat"]
print(cat_hash)

# Look up the cat_hash to get the string
cat_string = nlp.vocab.strings[cat_hash]
print(cat_string)

5439657043933447811
cat


In [16]:
from spacy.lang.en import English
nlp = English()
doc = nlp("David Bowie is a PERSON")
# Look up the hash for the string label "PERSON"
person_hash = nlp.vocab.strings["PERSON"]
print(person_hash)
# Look up the person_hash to get the string
person_string = nlp.vocab.strings[person_hash]
print(person_string)

380
PERSON


In [17]:
# #from spacy.lang.en import English
# from spacy.lang.de import German

# # Create an English and German nlp object
# nlp = English()
# nlp_de = German()

# # Get the ID for the string 'Bowie'
# bowie_id = nlp.vocab.strings["Bowie"]
# print(bowie_id)

# # Look up the ID for "Bowie" in the vocab
# print(nlp_de.vocab.strings[bowie_id])

# errors! : The string "Bowie" isn’t in the German vocab, so the hash can’t be resolved in the string store.
# never see it, errors
# Hashes can’t be reversed. To prevent this problem, add the word to the new vocab by processing a text or looking up the string, or use the same vocab to resolve the hash back to a string.